In [274]:
import sympy as sp
from IPython.display import Math

# Utility functions

In [275]:
def scale_to_clear_fractions(vec):
    """Scales vector to eliminate fractions (rational denominators)."""
    denominators = [term.as_numer_denom()[1] for term in vec]
    lcm = sp.lcm(denominators)
    return sp.simplify(lcm * vec)


In [276]:
def display_general_solution_factored(x_t, domain='real', x_symbol='x', symbol_index=None):
    """
    Display the general solution of a linear system of ODEs,
    factoring it as a sum of constants times vector expressions,
    and separating out any inhomogeneous term.

    Parameters:
        x_t         : SymPy column vector (Matrix with shape n×1)
        domain      : 'real', 'complex', or None — declares the domain of the constants
        x_symbol    : solution symbol, e.g., 'x'
        symbol_index: optional subscript index, e.g., 1 to produce x₁(t)
    """
    if not isinstance(x_t, sp.MatrixBase) or x_t.shape[1] != 1:
        raise TypeError("x_t must be a SymPy column vector (n×1 Matrix)")

    t = sp.Symbol('t', real=True)
    expr = x_t.expand()

    # Identify free variables that look like constants (e.g., c1, c2, ...)
    all_syms = expr.free_symbols
    ci_syms = sorted([s for s in all_syms if s.name.startswith('c') and s != t], key=lambda s: s.name)

    terms = []
    constants = set()
    remaining = expr.copy()

    for ci in ci_syms:
        # Extract the coefficient vector of ci
        vi = expr.applyfunc(lambda entry: sp.collect(entry, ci).coeff(ci))
        if vi != sp.zeros(*x_t.shape):
            terms.append(f"{sp.latex(ci)} {sp.latex(vi)}")
            constants.add(ci)
            # Subtract this term from remaining to isolate the inhomogeneous part
            remaining -= ci * vi

    # Check if any part remains (i.e., particular solution)
    if not remaining.equals(sp.zeros(*x_t.shape)):
        terms.append(sp.latex(remaining))

    # Format display
    prefix = f"{x_symbol}_{{{symbol_index}}}(t)" if symbol_index is not None else f"{x_symbol}(t)"
    latex_expr = prefix + " = " + " + ".join(terms)

    if domain is not None and constants:
        constant_list = ", ".join(sp.latex(c) for c in sorted(constants, key=lambda s: s.name))
        number_set = r"\mathbb{R}" if domain == 'real' else r"\mathbb{C}"
        latex_expr += rf", \quad \text{{where }} {constant_list} \in {number_set}"

    display(Math(latex_expr))


# Systems of 1st order differential equations

## Homogenous system

In [278]:
def construct_fundamental_matrix(A, tau_vals=None, verbose=True):
    """
    Construct the fundamental matrix solution Φ(t) to the system x'(t) = A x(t),
    using:
      - Theorem 2.7(a) for simple real eigenvalues
      - Theorem 2.7(b) for simple complex eigenvalues
      - Theorem 2.11 (special case) for eigenvalues with algebraic multiplicity 2 and geometric multiplicity 1
    """
    t = sp.symbols('t', real=True)
    I_n = sp.eye(A.shape[0])
    cols = []
    if verbose:
        display(Math(r"\text{Constructing fundamental matrix } \Phi(t) \text{ for the system } x'(t) = A x(t)"))
        display(Math(r"A = " + sp.latex(A)))
        char_poly = A.charpoly()
        display(Math(r"\text{Characteristic polynomial: } " + sp.latex(char_poly)))
        display(Math(r"\text{Characteristic polynomial factored: } " + sp.latex(char_poly.as_expr().factor())))

    if verbose:
      print("Iterating over eigenvalues and eigenvectors:")
    for eigenvalue, algebraic_mult, eigenvectors in A.eigenvects():
        geometric_mult = len(eigenvectors)

        if eigenvalue.is_real:
            if algebraic_mult == 1:
                # Theorem 2.7(a)
                v = eigenvectors[0]
                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Eigenvector = {v}. Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)")
                cols.append(sp.exp(eigenvalue * t) * v)

            elif algebraic_mult == 2 and geometric_mult == 1:
                # Theorem 2.11(a)
                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Eigenvector = {eigenvectors[0]}. Algebraic multiplicity = 2. Geometric multiplicity = 1. Using Theorem 2.11(a)")

                b11 = eigenvectors[0]
                J = A - eigenvalue * I_n
                b21, tau = J.gauss_jordan_solve(b11)

                # Set all free parameters
                if tau_vals is not None:
                    b21 = b21.subs({s: tau_vals for s in tau.free_symbols})

                x1 = sp.exp(eigenvalue * t) * b21
                x2 = sp.exp(eigenvalue * t) * (b21 + t * b11)

                if verbose:
                  display(Math(r"J = A - \lambda I = " + sp.latex(J)))
                  
                  # Display solutions
                  display(Math(fr"b_{{11}} = {sp.latex(b11)}"))

                  display(Math(r"\text{find } b_{ 21 } \text{ by solving } J b_{ 21 } = b_{22} \text{ for } b_{ 22 } = b_{ 11 }"))
                  display(Math(fr"b_{{21}} = {sp.latex(b21)}"))
                  display(Math(fr"b_{{22}} = {sp.latex(b11)}"))
                  display(Math(fr"x_{{1}} = {sp.latex(x1)}"))
                  display(Math(fr"x_{{2}} = {sp.latex(x2)}"))

                cols.extend([x1, x2])

        else:
            if sp.im(eigenvalue) < 0:
                continue  # Skip complex conjugate

            v = eigenvectors[0]
            J = A - eigenvalue * I_n

            if algebraic_mult == 1:
                # Theorem 2.7(b)
                complex_sol = sp.exp(eigenvalue * t) * v
                x1 = sp.re(complex_sol)
                x2 = sp.im(complex_sol)

                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Eigenvector = {v}. Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(b)")
                    display(Math(fr"complex \, solution = {sp.latex(complex_sol)}"))
      
                    display(Math(fr"x_{{1}} = {sp.latex(x1)}"))
                    display(Math(fr"x_{{2}} = {sp.latex(x2)}"))

                cols.extend([x1, x2])

            elif algebraic_mult == 2 and geometric_mult == 1:
                # Extended complex eigenvalue case
                b2 = v
                b1, tau = J.gauss_jordan_solve(b2)

                if tau_vals is not None:
                    b1 = b1.subs({s: tau_vals for s in tau.free_symbols})
                else:
                    b1 = b1.subs({s: 0 for s in tau.free_symbols})

                x1 = sp.exp(eigenvalue * t) * b2
                x2 = sp.exp(eigenvalue * t) * (b1 + t * b2)

                real_x1 = sp.re(x1)
                imag_x1 = sp.im(x1)
                real_x2 = sp.re(x2)
                imag_x2 = sp.im(x2)

                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Algebraic multiplicity = 2. Geometric multiplicity = 1. Using extended Theorem 2.7(b)")

                cols.extend([real_x1, imag_x1, real_x2, imag_x2])

    fundamental_matrix = sp.Matrix.hstack(*cols)
    fundamental_matrix = sp.Matrix.hstack(*[scale_to_clear_fractions(fundamental_matrix.col(i)) for i in range(fundamental_matrix.shape[1])])

    if verbose:
        display(Math(r"\Phi(t) = " + sp.latex(fundamental_matrix)))

    # return fundamental_matrix 
    return fundamental_matrix


## Inhomogeneous system

In [279]:
def solve_inhomogeneous_linear_system(A, u, x0, t0_val=0, tau_vals=None, verbose=True):
    """
    Solves the inhomogeneous system x'(t) = A x(t) + u with initial condition x(t0) = x0
    using Theorem 2.20 (parts i and ii).

    Parameters:
        A        : system matrix (SymPy Matrix)
        u        : input vector (SymPy column Matrix)
        x0       : initial condition vector (SymPy column Matrix)
        t0_val   : value of t0 (default 0)
        tau_vals : dict of free parameter substitutions (default None)
        verbose  : if True, display steps

    Returns:
        general_solution    : symbolic expression for x(t)
        particular_solution : symbolic expression for x_p(t)
    """
    t = sp.symbols('t', real=True)
    t0 = sp.symbols('t0', real=True)

    if verbose:
        display(Math(r"A = " + sp.latex(A)))
        display(Math(r"u = " + sp.latex(u)))
        display(Math(r"x_0 = " + sp.latex(x0)))
        display(Math(r"t_0 = " + sp.latex(t0_val)))

        display(Math(r"\text{General solution: } x(t) = \Phi(t) \bold{c} + \Phi(t) \int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt"))

    fundamental_matrix = construct_fundamental_matrix(A, tau_vals=tau_vals, verbose=True)
    inv_fundamental_matrix = fundamental_matrix.inv()
    inv_fundamental_matrix_u = inv_fundamental_matrix * u

    integral = sp.integrate(inv_fundamental_matrix_u, (t, t0, t))
    if t0_val is not None:
        integral = integral.subs(t0, t0_val)
    integral_term = fundamental_matrix * integral

    if verbose:
        display(Math(r"\Phi(t) = " + sp.latex(fundamental_matrix)))
        display(Math(r"\Phi(t)^{-1} = " + sp.latex(inv_fundamental_matrix)))
        display(Math(r"\Phi(t)^{-1} \bold{u} = " + sp.latex(inv_fundamental_matrix_u)))
        display(Math(r"\int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt = " + sp.latex(integral)))
        display(Math(r"\Phi(t) \int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt = " + sp.latex(integral_term)))

    # General solution
    c = sp.Matrix([sp.symbols(f'c{i+1}', real=True) for i in range(A.shape[1])])
    general_solution = fundamental_matrix * c + integral_term
    if verbose:
        display_general_solution_factored(general_solution, domain='real', x_symbol='x', symbol_index=None)

    # Particular solution (Theorem 2.20 ii)
    if x0 is not None:
      Φ_t0_inv = fundamental_matrix.subs(t, t0_val).inv()
      term1 = fundamental_matrix * Φ_t0_inv * x0
      term2 = integral_term.subs(t0, t0_val)
      particular_solution = term1 + term2

      if verbose:
          display(Math(r"\text{Particular solution: } x_p(t) = \Phi(t)\Phi(t_0)^{-1}\bold{x}_0 + \Phi(t) \int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt"))
          display(Math(r"\Phi(t) \Phi(t_0)^{-1} \bold{x}_0 = " + sp.latex(term1)))
          display_general_solution_factored(particular_solution, domain='real', x_symbol='x', symbol_index=None)
      return general_solution, particular_solution

    return general_solution


# Week 2

In [281]:
# 409,i)
A = sp.Matrix([[5, 3], [9, -1]]) 
sol = construct_fundamental_matrix(A, verbose=True, tau_vals=None) 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Iterating over eigenvalues and eigenvectors:
Eigenvalue = -4. Eigenvector = Matrix([[-1/3], [1]]). Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)
Eigenvalue = 8. Eigenvector = Matrix([[1], [1]]). Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)


<IPython.core.display.Math object>

In [282]:
# 409,ii)
A = sp.Matrix([[5, 3], [9, -1]])
t = sp.symbols('t', real=True)
u = sp.Matrix([[3],[-5]])*sp.exp(4*t)
sol = solve_inhomogeneous_linear_system(A, u, x0=None, t0_val=0, tau_vals=None, verbose=True)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [285]:
# 411
A = sp.Matrix([[0, 0, 1], [2, 0, -1],[-1,0,0]])
sol = construct_fundamental_matrix(A, verbose=True, tau_vals=None)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Iterating over eigenvalues and eigenvectors:
Eigenvalue = 0. Eigenvector = Matrix([[0], [1], [0]]). Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)
Eigenvalue = I. Eigenvector = Matrix([[-I], [-2 + I], [1]]). Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(b)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [286]:
# 431
A = sp.Matrix([[1, 2], [2, 4]])
u = sp.Matrix([1, 1])
x0 = sp.Matrix([[1],[0]])
sol = solve_inhomogeneous_linear_system(A, u, x0=x0, t0_val=0, tau_vals=None, verbose=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Week 3